# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集

訓練一個 CNN 模型來學習門牌號碼資料集。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux.Optimise: Optimiser, WeightDecay
using Flux: onehotbatch, onecold, logitcrossentropy
using Statistics, Random
import ProgressMeter
import MLDatasets
using CUDAapi

In [2]:
η = 5e-4             # learning rate
λ = 5e-4             # L2 regularizer param, implemented as weight decay
epochs = 60          # number of epochs
batchsize = 256      # batch size

256

## 使用CUDA

In [3]:
use_cuda = CUDAapi.has_cuda_gpu()
if use_cuda
    device = gpu
    println("Training on GPU")
else
    device = cpu
    println("Training on CPU")
end

Training on GPU


## 讀取資料

In [4]:
function get_data(batchsize=256)
    xtrain, ytrain = MLDatasets.SVHN2.traindata(Float32, 1:20000)
    xtest , ytest  = MLDatasets.SVHN2.testdata(Float32, 1:2000)

    #xtrain = reshape(xtrain, 28, 28, 1, :)
    #xtest  = reshape(xtest, 28, 28, 1, :)

    ytrain = onehotbatch(ytrain, 1:10)
    ytest  = onehotbatch(ytest, 1:10)

    train_loader = DataLoader(xtrain, ytrain, batchsize=batchsize, shuffle=true)
    test_loader  = DataLoader(xtest, ytest,  batchsize=batchsize)
    
    return train_loader, test_loader
end

train_loader, test_loader = get_data(batchsize);

## CNN 模型

In [5]:
# write your model here
num_params(model) = sum(length, Flux.params(model))

function buildModel(imgsize=(32,32,3))
#     return Chain(
#         x -> reshape(x, imgsize..., :),
#         Conv((5, 5), 3=>16, pad=(1,1), relu),
#         MaxPool((2,2)),
#         Conv((3, 3), 16=>32, pad=(1,1), relu),
#         MaxPool((2,2)),
#         Conv((3, 3), 32=>64, pad=(1,1), relu),
#         MaxPool((2,2)),
#         flatten,
#         Dense(576, 10),
#         softmax
#     )
    return Chain(
        x -> reshape(x, imgsize..., :),
        Conv((5, 5), 3=>24, stride=2),
        BatchNorm(24, relu),
        Conv((1, 1), 24=>24, stride=1),
        BatchNorm(24, relu),
        Conv((3, 3), 24=>48, stride=2),
        BatchNorm(48, relu),
        Conv((1, 1), 48=>48, stride=1),
        BatchNorm(48, relu),
        Conv((3, 3), 48=>96, stride=2),
        BatchNorm(96, relu),
        Conv((1, 1), 96=>96, stride=1),
        BatchNorm(96, relu),
        flatten,
        Dense(384, 10),
        softmax
    )
end

model = buildModel() |> device;
println("CNNs model: $(num_params(model)) trainable params");

CNNs model: 70594 trainable params


In [6]:
fake_input = randn((32, 32, 3)) |> device
fake_output = model(fake_input)

10×1 CuArrays.CuArray{Float32,2,Nothing}:
 0.098520525
 0.10139884
 0.11073899
 0.10396347
 0.10483897
 0.09984982
 0.1035858
 0.09728305
 0.08590987
 0.093910694

### Loss function

In [7]:
loss(ŷ, y) = logitcrossentropy(ŷ, y)

round4(x) = round(x, digits=4)

function calc_loss_accuracy(loader, model, device)
    l = 0f0
    acc = 0
    ntot = 0
    for (x, y) in loader
        x, y = x |> device, y |> device
        ŷ = model(x)
        l += loss(ŷ, y) * size(x)[end]        
        acc += sum(onecold(ŷ |> cpu) .== onecold(y |> cpu))
        ntot += size(x)[end]
    end
    return (loss = l/ntot |> round4, acc = acc/ntot*100 |> round4)
end

calc_loss_accuracy (generic function with 1 method)

## Callback 函式

In [8]:
# callback function
function callback(epoch)
    train = calc_loss_accuracy(train_loader, model, device)
    test = calc_loss_accuracy(test_loader, model, device)        
    println("Epoch: $epoch   Train: $(train)   Test: $(test)")
end

callback (generic function with 1 method)

## Training

In [9]:
# define optimizer
opt = ADAM(η) 
if λ > 0 
    opt = Optimiser(opt, WeightDecay(λ))
end

Optimiser(Any[ADAM(0.0005, (0.9, 0.999), IdDict{Any,Any}()), WeightDecay(0.0005)])

In [10]:
params_model = Flux.params(model)

println("Starting Training")
callback(0)

for epoch in 1:epochs
    progress = ProgressMeter.Progress(length(train_loader))
    
    # update parameters
    for (x, y) in train_loader
        x = x |> device
        y = y |> device
        grads_model = Flux.gradient(params_model) do
            ŷ = model(x)
            loss(ŷ, y)
        end
        Flux.Optimise.update!(opt, params_model, grads_model)
        ProgressMeter.next!(progress)   # comment out for no progress bar
    end
    
    #  logging
    if epoch % 5 == 0
        callback(epoch)
    end
end

Starting Training
Epoch: 0   Train: (loss = 2.302f0, acc = 14.76)   Test: (loss = 2.302f0, acc = 16.0)


Progress: 100%|█████████████████████████████████████████| Time: 0:00:45
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Epoch: 5   Train: (loss = 1.7356f0, acc = 75.125)   Test: (loss = 1.7874f0, acc = 69.55)

Progress:   6%|███                                      |  ETA: 0:00:02

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


...

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Epoch: 55   Train: (loss = 1.5399f0, acc = 94.36)   Test: (loss = 1.677f0, acc = 79.95)

Progress:   8%|████                                     |  ETA: 0:00:01

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Epoch: 60   Train: (loss = 1.5504f0, acc = 93.515)   Test: (loss = 1.6679f0, acc = 81.75)


### Evaluation

In [11]:
test = calc_loss_accuracy(test_loader, model, device)        
println("Test: $(test)")

Test: (loss = 1.6679f0, acc = 81.75)
